In [ ]:
import requests
import os
import time
from dotenv import load_dotenv

# ✅ STEP 0: 환경 세팅
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

search_url = "https://tools.kinds.or.kr/search/news"
feature_url = "https://tools.kinds.or.kr/feature"
trend_url = "https://tools.kinds.or.kr/time_line"
word_url = "https://tools.kinds.or.kr/word_cloud"

# ✅ STEP 1: 뉴스 수집 (키워드 중심 이슈 분석)
query = "인공지능 AND 산업 AND 정책"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 10

print("\n📥 STEP 1: 뉴스 수집 중...")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": fetch_count,
        "return_from": 0,
        "fields": ["title", "news_id"]
    }
}
search_resp = requests.post(search_url, json=search_payload)
docs = search_resp.json()["return_object"]["documents"]
news_ids = [doc["news_id"] for doc in docs]

# ✅ STEP 2: 본문 확보 + 키워드 추출
print("\n🔍 STEP 2: 키워드 추출 중...")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

for i, article in enumerate(articles, 1):
    title = article["title"]
    content = article["content"]
    print(f"\n{i}. 📰 {title}")

    # 특성 추출 API 호출
    feature_payload = {
        "access_key": API_KEY,
        "argument": {
            "title": title,
            "sub_title": "",
            "content": content
        }
    }
    try:
        f_resp = requests.post(feature_url, json=feature_payload)
        if f_resp.status_code == 200:
            raw_text = f_resp.json()["return_object"]["result"].get("title", "")
            keywords = [k.split("|")[0] for k in raw_text.split(" ") if "|" in k]
            print(f"🔑 키워드: {', '.join(keywords[:5])}")
    except:
        print("❌ 키워드 추출 실패")

# ✅ STEP 3: 이슈 맵 - 연관어 분석
print("\n🕸 STEP 3: 이슈 맵 분석 (연관 키워드 Top 10)")
word_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": "인공지능",
        "published_at": {
            "from": from_date,
            "until": until_date
        }
    }
}
word_resp = requests.post(word_url, json=word_payload).json()
word_list = sorted(word_resp["return_object"]["nodes"], key=lambda x: x["weight"], reverse=True)
for i, word in enumerate(word_list[:10], 1):
    print(f"{i}. 🔗 {word['name']} (연관도: {word['weight']:.2f})")

# ✅ STEP 4: 트렌드 분석 - 이슈 흐름
print("\n📈 STEP 4: 이슈 흐름 분석 (AI 트렌드 언급량)")
trend_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": "인공지능",
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "interval": "day",
        "normalize": "false"
    }
}
trend_resp = requests.post(trend_url, json=trend_payload).json()
time_line = trend_resp["return_object"]["time_line"]
for t in time_line:
    print(f"{t['label']} → {t['hits']}건")


📥 STEP 1: 뉴스 수집 중...

🔍 STEP 2: 키워드 추출 중...

1. 📰 'M7' 지고 'T10' 뜬다…中 펀드, 한달 새 3600억 원 유입
🔑 키워드: 3600억, 3600억_원_유입, T10, 펀드, 유입

2. 📰 부산시, 네이버클라우드와 AI 지능행정 미래 그린다
🔑 키워드: 네이버클라우드, AI, AI_지능행정_미래, AI_지능행정, 부산시

3. 📰 대선 출사표 던진 이철우…"박정희 정신 계승할 것"
🔑 키워드: 출사표, 이철우, 박정희, 정신_계승, 대선_출사표

4. 📰 “조선업, 경기 넘은 구조적 호황…중장기 주가 상승” [마켓시그널]
🔑 키워드: 중장기, 주가_상승, 구조적_호황, 조선업, 마켓시그널

5. 📰 이재명, 대선 첫 일정은 퓨리오사AI 방문…"성장경제 행보·AI 비전 제시"
🔑 키워드: AI, 성장경제, 퓨리오사, 퓨리오사AI_방문, 비전_제시

6. 📰 독일 하노버에서 경남 6개 기업 기술력 입증
🔑 키워드: 경남, 경남_6개, 하노버, 독일_하노버, 독일

7. 📰 부산시, 벤처·창업기업 ESG 선도기업 모집…사회가치경영 지원 강화
🔑 키워드: 창업기업, 사회가치경영, ESG, ESG_선도기업_모집, ESG_선도기업

8. 📰 與 "대선 공약에 AI 산업 경쟁력 강화 방안 반영할 것"
🔑 키워드: 경쟁력, AI, AI_산업, 경쟁력_강화, 방안_반영

9. 📰 30돌 맞은 서울모빌리티쇼 개막…"앞으로 30년, 모빌리티 혁명이 중심"
🔑 키워드: 서울모빌리티쇼, 모빌리티, 모빌리티_혁명, 서울모빌리티쇼_개막, 30돌

10. 📰 최상목 "'필수추경' 산불 집중 지원…AI·내수에 각 3~4조원 투입"
🔑 키워드: 산불_집중, 산불_집중_지원, 최상목, 최상목_필수추경, AI

🕸 STEP 3: 이슈 맵 분석 (연관 키워드 Top 10)
1. 🔗 AI (연관도: 180.00)
2. 🔗 반도체 (연관도: 7.06)
3. 🔗 삼성전자 (연관도: 6.00)
4. 🔗 중국 (연관도: 5.33)
5. 🔗 챗GPT (연관도

In [3]:
import requests
import os
import time
from dotenv import load_dotenv

# ✅ STEP 0: 환경 세팅
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

search_url = "https://tools.kinds.or.kr/search/news"
feature_url = "https://tools.kinds.or.kr/feature"
trend_url = "https://tools.kinds.or.kr/time_line"
word_url = "https://tools.kinds.or.kr/word_cloud"

# ✅ STEP 1: 뉴스 수집 (키워드 중심 이슈 분석)
query = "인공지능 OR 산업 OR 정책"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 10

print("\n📥 STEP 1: 뉴스 수집 중...")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": fetch_count,
        "return_from": 0,
        "fields": ["title", "news_id"]
    }
}
search_resp = requests.post(search_url, json=search_payload)
docs = search_resp.json()["return_object"]["documents"]
news_ids = [doc["news_id"] for doc in docs]

# ✅ STEP 2: 본문 확보 + 키워드 추출
print("\n🔍 STEP 2: 키워드 추출 중...")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

for i, article in enumerate(articles, 1):
    title = article["title"]
    content = article["content"]
    print(f"\n{i}. 📰 {title}")

    # 특성 추출 API 호출
    feature_payload = {
        "access_key": API_KEY,
        "argument": {
            "title": title,
            "sub_title": "",
            "content": content
        }
    }
    try:
        f_resp = requests.post(feature_url, json=feature_payload)
        if f_resp.status_code == 200:
            raw_text = f_resp.json()["return_object"]["result"].get("title", "")
            keywords = [k.split("|")[0] for k in raw_text.split(" ") if "|" in k]
            print(f"🔑 키워드: {', '.join(keywords[:5])}")
    except:
        print("❌ 키워드 추출 실패")

# ✅ STEP 3: 이슈 맵 - 연관어 분석
print("\n🕸 STEP 3: 이슈 맵 분석 (연관 키워드 Top 10)")
word_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": "인공지능",
        "published_at": {
            "from": from_date,
            "until": until_date
        }
    }
}
word_resp = requests.post(word_url, json=word_payload).json()
word_list = sorted(word_resp["return_object"]["nodes"], key=lambda x: x["weight"], reverse=True)
for i, word in enumerate(word_list[:10], 1):
    print(f"{i}. 🔗 {word['name']} (연관도: {word['weight']:.2f})")

# ✅ STEP 4: 트렌드 분석 - 이슈 흐름
print("\n📈 STEP 4: 이슈 흐름 분석 (AI 트렌드 언급량)")
trend_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": "인공지능",
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "interval": "day",
        "normalize": "false"
    }
}
trend_resp = requests.post(trend_url, json=trend_payload).json()
time_line = trend_resp["return_object"]["time_line"]
for t in time_line:
    print(f"{t['label']} → {t['hits']}건")


📥 STEP 1: 뉴스 수집 중...

🔍 STEP 2: 키워드 추출 중...

1. 📰 가천대 인공지능학과 졸업생들, 후배위해 대학 발전기금 1900만원 쾌척
🔑 키워드: 발전기금, 가천대, 대학_발전기금, 대학_발전기금_1900만, 가천대_인공지능학과

2. 📰 인천시 ‘미래산업 포럼’ 개최…제조업 디지털 전환 모색
🔑 키워드: 인천시, 인천시_미래산업, 디지털_전환, 인천시_미래산업_포럼, 디지털_전환_모색

3. 📰 카카오모빌리티, 자율주행 AI 데이터 기술 개발로 과기부장관상
🔑 키워드: 자율주행, 과기부, 과기부장관상, 자율주행_AI, AI

4. 📰 [투자의 창] 엔비디아 GTC 2025 탐방기
🔑 키워드: 엔비디아, 엔비디아_GTC, GTC, 탐방기, 투자

5. 📰 인공지능 중심도시 광주, ‘AI 대변인’ 도입
🔑 키워드: 중심도시, 대변인, 인공지능_중심도시, 인공지능_중심도시_광주, AI

6. 📰 수원하이텍고교 마이스터고 재도약 사업 선정
🔑 키워드: 재도약, 마이스터고, 수원하이텍고교, 재도약_사업, 수원하이텍고교_마이스터고

7. 📰 독일 하노버에서 경남 6개 기업 기술력 입증
🔑 키워드: 경남, 경남_6개, 하노버, 독일_하노버, 독일

8. 📰 "AI와 예술가의 만남" 국립광주과학관이 마련한 인공지능 융합콘서트
🔑 키워드: 인공지능, 예술가, 국립광주과학관, 인공지능_융합콘서트, 융합

9. 📰 부산시, 공무원 데이터·AI 인재양성 교육 확대… 행정 혁신 가속화
🔑 키워드: 인재양성, 인재양성_교육, 인재양성_교육_확대, 공무원, 행정_혁신

10. 📰 30돌 맞은 서울모빌리티쇼 개막…"앞으로 30년, 모빌리티 혁명이 중심"
🔑 키워드: 서울모빌리티쇼, 모빌리티, 모빌리티_혁명, 서울모빌리티쇼_개막, 30돌

🕸 STEP 3: 이슈 맵 분석 (연관 키워드 Top 10)
1. 🔗 AI (연관도: 180.00)
2. 🔗 반도체 (연관도: 7.06)
3. 🔗 삼성전자 (연관도: 6.00)
4. 🔗 중국 (연관도: 5.33)
5. 🔗 챗

# clustering

In [7]:
import requests
import os
import time
from dotenv import load_dotenv
from collections import Counter, defaultdict

# ✅ STEP 0: 환경 세팅
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

search_url = "https://tools.kinds.or.kr/search/news"
feature_url = "https://tools.kinds.or.kr/feature"

# ✅ STEP 1: 뉴스 수집
query = "인공지능 AND 산업 AND 정책"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 30

print("\n📥 STEP 1: 뉴스 수집 중...")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": fetch_count,
        "return_from": 0,
        "fields": ["title", "news_id", "published_at"]
    }
}
search_resp = requests.post(search_url, json=search_payload)
docs = search_resp.json()["return_object"]["documents"]
news_ids = [doc["news_id"] for doc in docs]
titles = [doc["title"] for doc in docs]
dates = [doc["published_at"][:10] for doc in docs]  # YYYY-MM-DD

# ✅ STEP 2: 본문 조회 + 키워드 추출
print("\n🔍 STEP 2: 키워드 추출 중...")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

full_articles = []
all_keywords = []

for i, article in enumerate(articles):
    title = article["title"]
    content = article["content"]
    date = dates[i]

    # 특성 추출
    feature_payload = {
        "access_key": API_KEY,
        "argument": {
            "title": title,
            "sub_title": "",
            "content": content
        }
    }
    f_resp = requests.post(feature_url, json=feature_payload)
    keyword_list = []
    if f_resp.status_code == 200:
        try:
            raw = f_resp.json()["return_object"]["result"]["title"]
            keyword_list = [k.split("|")[0] for k in raw.split(" ") if "|" in k]
            all_keywords.extend(keyword_list)
        except:
            pass

    full_articles.append({
        "title": title,
        "date": date,
        "keywords": keyword_list,
        "content": content
    })

# ✅ STEP 3: 전체 키워드 분석 + 기사 그룹화
print("\n📊 STEP 3: 이슈 그룹 키워드 분석")
counter = Counter(all_keywords)
top_keywords = [kw for kw, _ in counter.most_common(5)]
print("Top 키워드:", ", ".join(top_keywords))

# 기사 그룹핑
grouped_articles = {k: [] for k in top_keywords}
for article in full_articles:
    for k in top_keywords:
        if k in article["keywords"]:
            grouped_articles[k].append(article)

# ✅ STEP 4: 날짜 기반 흐름 분석
print("\n📈 STEP 4: 날짜별 흐름 요약")
date_map = defaultdict(list)
for art in full_articles:
    date_map[art["date"]].append(art)

for date in sorted(date_map.keys()):
    daily = date_map[date]
    d_keywords = []
    for a in daily:
        d_keywords.extend(a["keywords"])
    top_d = Counter(d_keywords).most_common(3)
    print(f"\n📅 {date} ({len(daily)}건)")
    print("- 주요 키워드:", ", ".join([k for k, _ in top_d]))
    print("- 대표 기사:", daily[0]["title"][:60], "...")
    print("- 전체 기사:")
    for art in daily:
        print("  •", art["title"])


📥 STEP 1: 뉴스 수집 중...

🔍 STEP 2: 키워드 추출 중...

📊 STEP 3: 이슈 그룹 키워드 분석
Top 키워드: AI, 이재명, 대선, 기업, 로터리

📈 STEP 4: 날짜별 흐름 요약

📅 2025-04-02 (4건)
- 주요 키워드: AI, 30주년, 서울
- 대표 기사: [로터리] 30주년 맞은 서울 모빌리티쇼 ? ...
- 전체 기사:
  • [로터리] 30주년 맞은 서울 모빌리티쇼 ?
  • 부산시, 네이버클라우드와 AI 지능행정 미래 그린다
  • 30돌 맞은 서울모빌리티쇼 개막…"앞으로 30년, 모빌리티 혁명이 중심"
  • 최상목 "'필수추경' 산불 집중 지원…AI·내수에 각 3~4조원 투입"

📅 2025-04-03 (5건)
- 주요 키워드: AI, 산업, 총동원
- 대표 기사: “무역·수출 중기지원…정책금융 총동원해야” [S마켓 인사이드] ...
- 전체 기사:
  • “무역·수출 중기지원…정책금융 총동원해야” [S마켓 인사이드]
  • 'AI 기타국가' 된 한국
  • “조선업, 경기 넘은 구조적 호황…중장기 주가 상승” [마켓시그널]
  • 與 "대선 공약에 AI 산업 경쟁력 강화 방안 반영할 것"
  • 칩 확보서 데이터센터까지…전 산업에 'AI 고속도로' 깔아야

📅 2025-04-04 (2건)
- 주요 키워드: 대선, 피고인, 이재명
- 대표 기사: 김문수 "깨끗한 제가 피고인 이재명 이긴다"…21대 대선 출마 선언 ...
- 전체 기사:
  • 김문수 "깨끗한 제가 피고인 이재명 이긴다"…21대 대선 출마 선언
  • 대선 출사표 던진 이철우…"박정희 정신 계승할 것"

📅 2025-04-06 (1건)
- 주요 키워드: 인증제도, 간담회, 인증제도_개편방안_간담회_개최
- 대표 기사: 국표원, 신제품(NEP) 인증제도 개편방안 간담회 개최 ...
- 전체 기사:
  • 국표원, 신제품(NEP) 인증제도 개편방안 간담회 개최

📅 2025-04-07 (1건)
- 주요 키워드: 경제_MBTI, MBTI, 소

In [25]:
import os
import requests
import time
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

# ✅ STEP 0: 환경 설정
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
search_url = "https://tools.kinds.or.kr/search/news"

# ✅ STEP 1: 뉴스 수집
query = "인공지능 AND 산업 AND 정책"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 20

print("\n📥 뉴스 수집 중...")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": fetch_count,
        "return_from": 0,
        "fields": ["title", "news_id", "published_at"]
    }
}
search_resp = requests.post(search_url, json=search_payload)
docs = search_resp.json()["return_object"]["documents"]

news_ids = [doc["news_id"] for doc in docs]
titles = [doc["title"] for doc in docs]
dates = [doc["published_at"][:10] for doc in docs]

# ✅ STEP 2: 기사 본문 확보
print("\n📄 본문 수집 중...")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

texts = []
meta = []
for i, article in enumerate(articles):
    text = article["title"] + "\n" + article["content"]
    texts.append(text)
    meta.append({"title": article["title"], "date": dates[i]})

# ✅ STEP 3: TF-IDF 벡터화
print("\n🔍 텍스트 벡터화 중...")
vectorizer = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(texts)
terms = vectorizer.get_feature_names_out()

# ✅ STEP 4: KMeans 군집화
print("\n🧠 KMeans 기반 군집화...")
num_clusters = 4
km = KMeans(n_clusters=num_clusters, random_state=42)
km.fit(tfidf_matrix)
labels = km.labels_

# ✅ STEP 5: 결과 출력
print("\n📦 뉴스 군집화 결과:")
groups = {}
for i, label in enumerate(labels):
    if label not in groups:
        groups[label] = []
    groups[label].append({"title": meta[i]["title"], "text": texts[i]})

for label, articles in groups.items():
    print(f"\n🔹 클러스터 {label} ({len(articles)}건)")
    cluster_texts = [a["text"] for a in articles]
    cluster_matrix = vectorizer.transform(cluster_texts)
    summed = np.array(cluster_matrix.sum(axis=0)).flatten()
    top_indices = summed.argsort()[::-1][:5]
    top_keywords = [terms[i] for i in top_indices]
    print("- 주요 키워드:", ", ".join(top_keywords))
    for a in articles:
        print(f" - {a['title']}")


📥 뉴스 수집 중...

📄 본문 수집 중...

🔍 텍스트 벡터화 중...

🧠 KMeans 기반 군집화...

📦 뉴스 군집화 결과:

🔹 클러스터 0 (6건)
- 주요 키워드: ai, 한다는, 인재, 예산, 한다
 - 염재호 “2030년까지 정부 업무 95% AI 적용해야”
 - “무역·수출 중기지원…정책금융 총동원해야” [S마켓 인사이드]
 - 부산시, 네이버클라우드와 AI 지능행정 미래 그린다
 - 독일 하노버에서 경남 6개 기업 기술력 입증
 - 與 "대선 공약에 AI 산업 경쟁력 강화 방안 반영할 것"
 - 최상목 "'필수추경' 산불 집중 지원…AI·내수에 각 3~4조원 투입"

🔹 클러스터 2 (5건)
- 주요 키워드: 중국, 미국, 트럼프, ai, 있다
 - 'M7' 지고 'T10' 뜬다…中 펀드, 한달 새 3600억 원 유입
 - 'AI 기타국가' 된 한국
 - [사설] 기업 경영 위축시키는 ‘더 센’ 상법 개정 밀어붙일 때인가
 - 부산시, 벤처·창업기업 ESG 선도기업 모집…사회가치경영 지원 강화
 - 트럼프, ‘석탄 확대’ 행정명령… AI發 전력수요 급증 대비

🔹 클러스터 1 (7건)
- 주요 키워드: 모빌리티, 있다, 국가, 대한민국, 미래
 - 세계기자대회 참가 52개국 기자들 시흥 찾아…시화호의 기적 등 탐방
 - “이재명이 달라졌다”…‘AI·실용주의’ 전면에 내세운 집권 청사진 공개
 - 대선 출사표 던진 이철우…"박정희 정신 계승할 것"
 - “조선업, 경기 넘은 구조적 호황…중장기 주가 상승” [마켓시그널]
 - [로터리] 지역의 벚꽃엔딩, 이젠 끝내야
 - 30돌 맞은 서울모빌리티쇼 개막…"앞으로 30년, 모빌리티 혁명이 중심"
 - 국표원, 신제품(NEP) 인증제도 개편방안 간담회 개최

🔹 클러스터 3 (2건)
- 주요 키워드: 대표는, 이재명, ai, 대변인은, 일정으로
 - 이재명, 14일 '퓨리오사AI' 찾는다
 - 이재명, 대선 첫 일정은 퓨리오사AI 방문…"성장경제 행보·AI 비전 제시"


In [21]:
import requests
import os
import time
from dotenv import load_dotenv
from collections import Counter, defaultdict

# ✅ STEP 0: 환경 세팅
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

search_url = "https://tools.kinds.or.kr/search/news"
feature_url = "https://tools.kinds.or.kr/feature"

# ✅ STEP 1: 뉴스 수집
query = "인공지능 AND 산업 AND 정책"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 30

print("\n📥 STEP 1: 뉴스 수집 중...")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": fetch_count,
        "return_from": 0,
        "fields": ["title", "news_id", "published_at"]
    }
}
search_resp = requests.post(search_url, json=search_payload)
docs = search_resp.json()["return_object"]["documents"]
news_ids = [doc["news_id"] for doc in docs]
titles = [doc["title"] for doc in docs]
dates = [doc["published_at"][:10] for doc in docs]  # YYYY-MM-DD

# ✅ STEP 2: 본문 조회 + 키워드 추출
print("\n🔍 STEP 2: 키워드 추출 중...")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

full_articles = []
all_keywords = []

for i, article in enumerate(articles):
    title = article["title"]
    content = article["content"]
    date = dates[i]

    # 특성 추출
    feature_payload = {
        "access_key": API_KEY,
        "argument": {
            "title": title,
            "sub_title": "",
            "content": content
        }
    }
    f_resp = requests.post(feature_url, json=feature_payload)
    keyword_list = []
    if f_resp.status_code == 200:
        try:
            raw = f_resp.json()["return_object"]["result"]["title"]
            keyword_list = [k.split("|")[0] for k in raw.split(" ") if "|" in k]
            all_keywords.extend(keyword_list)
        except:
            pass

    full_articles.append({
        "title": title,
        "date": date,
        "keywords": keyword_list,
        "content": content
    })

# ✅ STEP 3: 전체 키워드 분석 + 기사 그룹화
print("\n📊 STEP 3: 이슈 그룹 키워드 분석")
counter = Counter(all_keywords)
top_keywords = [kw for kw, _ in counter.most_common(5)]
print("Top 키워드:", ", ".join(top_keywords))

# 기사 그룹핑
grouped_articles = {k: [] for k in top_keywords}
for article in full_articles:
    for k in top_keywords:
        if k in article["keywords"]:
            grouped_articles[k].append(article)

# ✅ STEP 4: 날짜 기반 흐름 분석
print("\n📈 STEP 4: 날짜별 흐름 요약")
date_map = defaultdict(list)
for art in full_articles:
    date_map[art["date"]].append(art)

for date in sorted(date_map.keys()):
    daily = date_map[date]
    d_keywords = []
    for a in daily:
        d_keywords.extend(a["keywords"])
    top_d = Counter(d_keywords).most_common(3)
    print(f"\n📅 {date} ({len(daily)}건)")
    print("- 주요 키워드:", ", ".join([k for k, _ in top_d]))
    print("- 대표 기사:", daily[0]["title"][:60], "...")
    print("- 전체 기사:")
    for art in daily:
        print("  •", art["title"])

    print("\n📝 기사 본문 요약:")
    for art in daily:
        print(f"[{art['title']}]")
        print(art['content'][:300], "...\n")


📥 STEP 1: 뉴스 수집 중...

🔍 STEP 2: 키워드 추출 중...

📊 STEP 3: 이슈 그룹 키워드 분석
Top 키워드: AI, 이재명, 대선, 기업, 로터리

📈 STEP 4: 날짜별 흐름 요약

📅 2025-04-02 (4건)
- 주요 키워드: AI, 30주년, 서울
- 대표 기사: [로터리] 30주년 맞은 서울 모빌리티쇼 ? ...
- 전체 기사:
  • [로터리] 30주년 맞은 서울 모빌리티쇼 ?
  • 부산시, 네이버클라우드와 AI 지능행정 미래 그린다
  • 30돌 맞은 서울모빌리티쇼 개막…"앞으로 30년, 모빌리티 혁명이 중심"
  • 최상목 "'필수추경' 산불 집중 지원…AI·내수에 각 3~4조원 투입"

📝 기사 본문 요약:
[[로터리] 30주년 맞은 서울 모빌리티쇼 ?]
한국의 자동차 산업은 많은 어려움이 있었지만 이를 극복하고 우리나라 경제성장을 이끄는 중추적인 역할을 하고 있다. 지금은 한 해 420만 대에 달하는 자동차를 생산하면서 2년 연속 900억 달러 수출을 돌파했다. 
 

 자동차 산업의 지형은 최근 빠르게 재편되고 있다. 전기차 시장의 확대는 물론 자율주행·커넥티비티·인공지능(AI)·로보틱스 등 각종 기술 융합과 함께 자동차는 ‘움직이는 기술 집약체’, 즉 모빌리티 산업으로 진화하고 있다. 또 모빌리티는 항공과 선박, 건설기계 등 여러 영역을 넘어서 변신을 거듭하고 있다. 
 

  ...

[부산시, 네이버클라우드와 AI 지능행정 미래 그린다]
부산시는 8일 시청에서 네이버클라우드와 인공지능(AI) 기술을 활용한 지능형 행정 시스템 구축하는 협약을 체결했다. 이날 협약은 공공분야 AI 행정혁신을 추진하기 위한 민관 협력 기반을 마련하고 지속 가능한 AI·데이터 기반 행정 구현을 목표로 한다. 
 

 협약에 따라 시는 정책적·행정적 지원과 행정혁신 프로젝트를 주도한다. 네이버클라우드는 기술 지원, 해법 개발, 지역업체와의 협업을 담당한다. 
 

 시는 올해 내부 행정 업무에 생성형 AI 서비스를 도입해 공

# ai 이슈 맵(오늘의 핵심 키워드)

In [22]:
import requests
import os
import time
from dotenv import load_dotenv
from collections import Counter, defaultdict

# ✅ STEP 0: 환경 세팅
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")

search_url = "https://tools.kinds.or.kr/search/news"
feature_url = "https://tools.kinds.or.kr/feature"

# ✅ STEP 1: 뉴스 수집
query = "인공지능 AND 산업 AND 정책"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 30

print("\n📥 STEP 1: 뉴스 수집 중...")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": fetch_count,
        "return_from": 0,
        "fields": ["title", "news_id", "published_at"]
    }
}
search_resp = requests.post(search_url, json=search_payload)
docs = search_resp.json()["return_object"]["documents"]
news_ids = [doc["news_id"] for doc in docs]
titles = [doc["title"] for doc in docs]
dates = [doc["published_at"][:10] for doc in docs]  # YYYY-MM-DD

# ✅ STEP 2: 본문 조회 + 키워드 추출
print("\n🔍 STEP 2: 키워드 추출 중...")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

full_articles = []
all_keywords = []

for i, article in enumerate(articles):
    title = article["title"]
    content = article["content"]
    date = dates[i]

    # 특성 추출
    feature_payload = {
        "access_key": API_KEY,
        "argument": {
            "title": title,
            "sub_title": "",
            "content": content
        }
    }
    f_resp = requests.post(feature_url, json=feature_payload)
    keyword_list = []
    if f_resp.status_code == 200:
        try:
            raw = f_resp.json()["return_object"]["result"]["title"]
            keyword_list = [k.split("|")[0] for k in raw.split(" ") if "|" in k]
            all_keywords.extend(keyword_list)
        except:
            pass

    full_articles.append({
        "title": title,
        "date": date,
        "keywords": keyword_list,
        "content": content
    })

# ✅ STEP 3: 전체 키워드 분석 + 기사 그룹화
print("\n📊 STEP 3: 이슈 그룹 키워드 분석")
counter = Counter(all_keywords)
top_keywords = [kw for kw, _ in counter.most_common(5)]
print("Top 키워드:", ", ".join(top_keywords))

# 기사 그룹핑
grouped_articles = {k: [] for k in top_keywords}
for article in full_articles:
    for k in top_keywords:
        if k in article["keywords"]:
            grouped_articles[k].append(article)

# ✅ STEP 4: 날짜 기반 흐름 분석
print("\n📈 STEP 4: 날짜별 흐름 요약")
date_map = defaultdict(list)
for art in full_articles:
    date_map[art["date"]].append(art)

for date in sorted(date_map.keys()):
    daily = date_map[date]
    d_keywords = []
    for a in daily:
        d_keywords.extend(a["keywords"])
    top_d = Counter(d_keywords).most_common(3)
    print(f"\n📅 {date} ({len(daily)}건)")
    print("- 주요 키워드:", ", ".join([k for k, _ in top_d]))
    print("- 대표 기사:", daily[0]["title"][:60], "...")
    print("- 전체 기사:")
    for art in daily:
        print("  •", art["title"])

    print("\n📝 기사 본문 요약:")
    for art in daily:
        print(f"[{art['title']}]")
        print(art['content'][:300], "...\n")

# ✅ STEP 5: 오늘의 핵심 키워드 (AI 이슈 맵)
print("\n🧠 오늘의 AI 핵심 키워드 맵")
for i, (kw, count) in enumerate(counter.most_common(5), 1):
    print(f"{i}. {kw} (언급 횟수: {count})")



📥 STEP 1: 뉴스 수집 중...

🔍 STEP 2: 키워드 추출 중...

📊 STEP 3: 이슈 그룹 키워드 분석
Top 키워드: AI, 이재명, 대선, 기업, 로터리

📈 STEP 4: 날짜별 흐름 요약

📅 2025-04-02 (4건)
- 주요 키워드: AI, 30주년, 서울
- 대표 기사: [로터리] 30주년 맞은 서울 모빌리티쇼 ? ...
- 전체 기사:
  • [로터리] 30주년 맞은 서울 모빌리티쇼 ?
  • 부산시, 네이버클라우드와 AI 지능행정 미래 그린다
  • 30돌 맞은 서울모빌리티쇼 개막…"앞으로 30년, 모빌리티 혁명이 중심"
  • 최상목 "'필수추경' 산불 집중 지원…AI·내수에 각 3~4조원 투입"

📝 기사 본문 요약:
[[로터리] 30주년 맞은 서울 모빌리티쇼 ?]
한국의 자동차 산업은 많은 어려움이 있었지만 이를 극복하고 우리나라 경제성장을 이끄는 중추적인 역할을 하고 있다. 지금은 한 해 420만 대에 달하는 자동차를 생산하면서 2년 연속 900억 달러 수출을 돌파했다. 
 

 자동차 산업의 지형은 최근 빠르게 재편되고 있다. 전기차 시장의 확대는 물론 자율주행·커넥티비티·인공지능(AI)·로보틱스 등 각종 기술 융합과 함께 자동차는 ‘움직이는 기술 집약체’, 즉 모빌리티 산업으로 진화하고 있다. 또 모빌리티는 항공과 선박, 건설기계 등 여러 영역을 넘어서 변신을 거듭하고 있다. 
 

  ...

[부산시, 네이버클라우드와 AI 지능행정 미래 그린다]
부산시는 8일 시청에서 네이버클라우드와 인공지능(AI) 기술을 활용한 지능형 행정 시스템 구축하는 협약을 체결했다. 이날 협약은 공공분야 AI 행정혁신을 추진하기 위한 민관 협력 기반을 마련하고 지속 가능한 AI·데이터 기반 행정 구현을 목표로 한다. 
 

 협약에 따라 시는 정책적·행정적 지원과 행정혁신 프로젝트를 주도한다. 네이버클라우드는 기술 지원, 해법 개발, 지역업체와의 협업을 담당한다. 
 

 시는 올해 내부 행정 업무에 생성형 AI 서비스를 도입해 공

In [24]:
import requests
import os
from dotenv import load_dotenv

# ✅ STEP 0: 환경 세팅
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
today_keyword_url = "https://tools.kinds.or.kr/today_category_keyword"

# ✅ STEP 5: 오늘의 AI 핵심 키워드 맵 (카테고리 기반 API 활용)
print("\n🧠 STEP 5: 오늘의 키워드 API 기반 이슈 맵")
today_payload = {
    "access_key": API_KEY,
    "argument": {}
}
today_response = requests.post(today_keyword_url, json=today_payload)

if today_response.status_code == 200:
    today_result = today_response.json()
    print("✅ 오늘의 키워드 API 호출 성공!\n")

    print("🔑 분야별 주요 키워드 (cate_keyword):\n")
    for item in today_result["return_object"].get("cate_keyword", []):
        if item["category_name"] == "IT_과학":
            print(f"📌 분야: {item['category_name']}")
            print(f"   🔍 키워드: {item['named_entity']}")
            print(f"   📊 등장 횟수: {item['named_entity_count']}")
            print(f"   🧠 타입: {item['named_entity_type']} | 단계: {item['entity_step']}")
            print("-" * 50)
else:
    print("❌ 오늘의 키워드 API 오류:", today_response.status_code)
    try:
        print("사유:", today_response.json().get("reason", "알 수 없는 오류"))
    except:
        print(today_response.text)


🧠 STEP 5: 오늘의 키워드 API 기반 이슈 맵
✅ 오늘의 키워드 API 호출 성공!

🔑 분야별 주요 키워드 (cate_keyword):

📌 분야: IT_과학
   🔍 키워드: 미국
   📊 등장 횟수: 135
   🧠 타입: LC | 단계: step4
--------------------------------------------------
📌 분야: IT_과학
   🔍 키워드: 한국
   📊 등장 횟수: 126
   🧠 타입: LC | 단계: step4
--------------------------------------------------
📌 분야: IT_과학
   🔍 키워드: 자원
   📊 등장 횟수: 71
   🧠 타입: OG | 단계: step3
--------------------------------------------------
📌 분야: IT_과학
   🔍 키워드: 일본
   📊 등장 횟수: 61
   🧠 타입: LC | 단계: step3
--------------------------------------------------
📌 분야: IT_과학
   🔍 키워드: 애플
   📊 등장 횟수: 60
   🧠 타입: OG | 단계: step3
--------------------------------------------------
📌 분야: IT_과학
   🔍 키워드: 중국
   📊 등장 횟수: 58
   🧠 타입: LC | 단계: step3
--------------------------------------------------
📌 분야: IT_과학
   🔍 키워드: 유럽
   📊 등장 횟수: 51
   🧠 타입: LC | 단계: step3
--------------------------------------------------
📌 분야: IT_과학
   🔍 키워드: 삼성전자
   📊 등장 횟수: 50
   🧠 타입: OG | 단계: step3
--------------------------------------------

In [26]:
import os
import requests
import time
from dotenv import load_dotenv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np

# ✅ STEP 0: 환경 설정
load_dotenv()
API_KEY = os.getenv("BIGKINDS_KEY")
search_url = "https://tools.kinds.or.kr/search/news"

# ✅ STEP 1: 뉴스 수집
query = "인공지능 AND 산업 AND 정책"
from_date = "2025-04-01"
until_date = "2025-04-14"
fetch_count = 20

print("\n📥 뉴스 수집 중...")
search_payload = {
    "access_key": API_KEY,
    "argument": {
        "query": query,
        "published_at": {
            "from": from_date,
            "until": until_date
        },
        "return_size": fetch_count,
        "return_from": 0,
        "fields": ["title", "news_id", "published_at"]
    }
}
search_resp = requests.post(search_url, json=search_payload)
docs = search_resp.json()["return_object"]["documents"]

news_ids = [doc["news_id"] for doc in docs]
titles = [doc["title"] for doc in docs]
dates = [doc["published_at"][:10] for doc in docs]

# ✅ STEP 2: 기사 본문 확보
print("\n📄 본문 수집 중...")
view_payload = {
    "access_key": API_KEY,
    "argument": {
        "news_ids": news_ids,
        "fields": ["title", "content"]
    }
}
view_resp = requests.post(search_url, json=view_payload)
articles = view_resp.json()["return_object"]["documents"]

texts = []
meta = []
for i, article in enumerate(articles):
    text = article["title"] + "\n" + article["content"]
    texts.append(text)
    meta.append({"title": article["title"], "date": dates[i]})

# ✅ STEP 3: TF-IDF 벡터화
print("\n🔍 텍스트 벡터화 중...")
vectorizer = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(texts)
terms = vectorizer.get_feature_names_out()

# ✅ STEP 4: KMeans 군집화
print("\n🧠 KMeans 기반 군집화...")
num_clusters = 4
km = KMeans(n_clusters=num_clusters, random_state=42)
km.fit(tfidf_matrix)
labels = km.labels_

# ✅ STEP 5: 결과 출력
print("\n📦 뉴스 군집화 결과:")
groups = {}
for i, label in enumerate(labels):
    if label not in groups:
        groups[label] = []
    groups[label].append({"title": meta[i]["title"], "text": texts[i]})

for label, articles in groups.items():
    print(f"\n🔹 클러스터 {label} ({len(articles)}건)")
    cluster_texts = [a["text"] for a in articles]
    cluster_matrix = vectorizer.transform(cluster_texts)
    summed = np.array(cluster_matrix.sum(axis=0)).flatten()
    top_indices = summed.argsort()[::-1][:5]
    top_keywords = [terms[i] for i in top_indices]
    print("- 주요 키워드:", ", ".join(top_keywords))
    for a in articles:
        print(f" - {a['title']}")


📥 뉴스 수집 중...

📄 본문 수집 중...

🔍 텍스트 벡터화 중...

🧠 KMeans 기반 군집화...

📦 뉴스 군집화 결과:

🔹 클러스터 0 (6건)
- 주요 키워드: ai, 한다는, 인재, 예산, 한다
 - 염재호 “2030년까지 정부 업무 95% AI 적용해야”
 - “무역·수출 중기지원…정책금융 총동원해야” [S마켓 인사이드]
 - 부산시, 네이버클라우드와 AI 지능행정 미래 그린다
 - 독일 하노버에서 경남 6개 기업 기술력 입증
 - 與 "대선 공약에 AI 산업 경쟁력 강화 방안 반영할 것"
 - 최상목 "'필수추경' 산불 집중 지원…AI·내수에 각 3~4조원 투입"

🔹 클러스터 2 (5건)
- 주요 키워드: 중국, 미국, 트럼프, ai, 있다
 - 'M7' 지고 'T10' 뜬다…中 펀드, 한달 새 3600억 원 유입
 - 'AI 기타국가' 된 한국
 - [사설] 기업 경영 위축시키는 ‘더 센’ 상법 개정 밀어붙일 때인가
 - 부산시, 벤처·창업기업 ESG 선도기업 모집…사회가치경영 지원 강화
 - 트럼프, ‘석탄 확대’ 행정명령… AI發 전력수요 급증 대비

🔹 클러스터 1 (7건)
- 주요 키워드: 모빌리티, 있다, 국가, 대한민국, 미래
 - 세계기자대회 참가 52개국 기자들 시흥 찾아…시화호의 기적 등 탐방
 - “이재명이 달라졌다”…‘AI·실용주의’ 전면에 내세운 집권 청사진 공개
 - 대선 출사표 던진 이철우…"박정희 정신 계승할 것"
 - “조선업, 경기 넘은 구조적 호황…중장기 주가 상승” [마켓시그널]
 - [로터리] 지역의 벚꽃엔딩, 이젠 끝내야
 - 30돌 맞은 서울모빌리티쇼 개막…"앞으로 30년, 모빌리티 혁명이 중심"
 - 국표원, 신제품(NEP) 인증제도 개편방안 간담회 개최

🔹 클러스터 3 (2건)
- 주요 키워드: 대표는, 이재명, ai, 대변인은, 일정으로
 - 이재명, 14일 '퓨리오사AI' 찾는다
 - 이재명, 대선 첫 일정은 퓨리오사AI 방문…"성장경제 행보·AI 비전 제시"
